### Log

In [1]:
import datetime, threading
from threading import Lock

module_version = "[1.00]"
log_file_path = "./log.txt"
_log_lock = Lock()

def LOGD(*args, **args2):
    with _log_lock:
        prefix = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S.%f') + " " + str(threading.get_ident()) + " " + module_version + '[D]'
        print(prefix, *args, **args2)
        with open(log_file_path, "a") as f:
            print(prefix, *args, **args2, file=f)

def LOGW(*args, **args2):
    with _log_lock:
        prefix = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S.%f') + " " + str(threading.get_ident()) + " " + module_version + '[W]'
        print(prefix, *args, **args2)
        with open(log_file_path, "a") as f:
            print(prefix, *args, **args2, file=f)

def LOGE(*args, **args2):
    with _log_lock:
        prefix = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S.%f') + " " + str(threading.get_ident()) + " " + module_version + '[E]'
        print(prefix, *args, **args2)
        with open(log_file_path, "a") as f:
            print(prefix, *args, **args2, file=f)

             # YEAR/MM/DD HH:MM:SS.UUUUUU PID   VER  TAG your_msg
LOGD("LOGD") # 2020/05/17 09:01:47.947437 2664 [1.00][D] LOGD
LOGW("LOGW") # 2020/05/17 09:01:47.949437 2664 [1.00][W] LOGW
LOGE("LOGE") # 2020/05/17 09:01:47.950437 2664 [1.00][E] LOGE

2020/05/17 21:12:32.256204 1480 [1.00][D] LOGD
2020/05/17 21:12:32.257204 1480 [1.00][W] LOGW
2020/05/17 21:12:32.259204 1480 [1.00][E] LOGE


### Server

In [2]:
import socket, select, threading, time, traceback

def server_thread():
    port = 10020
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
    server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server.bind(('localhost', port))
    server.listen(5)
    inputs = [server]
    while True:
        try:
            readable, _, _ = select.select(inputs, [], [])
            for sck in readable:
                if sck is server:
                    client, addr = sck.accept()
                    client.setblocking(0)
                    inputs.append(client)
                    LOGD("remote connected, client num=", len(inputs)-1)
                else:
                    try:
                        data = sck.recv(1024)
                        sck.send(b'server received: ' + data)
                    except Exception as err: # if remote side close the connection
                        sck.close()
                        inputs.remove(sck)
                        LOGD("remote disconnected, client num=", len(inputs)-1)
        except Exception as err:
            LOGE(traceback.format_exc())

threading.Thread(target=server_thread).start()

### Client

In [3]:
import socket

try:
    port = 10020
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    client.connect(('localhost', port))
    
    client.send(b'hello from client')
    data = client.recv(1024)
    LOGD(data)
    
    client.close()
except Error as err:
    LOGE(err)


2020/05/17 21:12:32.289206 3424 [1.00][D] remote connected, client num= 1
2020/05/17 21:12:32.299206 1480 [1.00][D] b'server received: hello from client'
2020/05/17 21:12:32.301206 3424 [1.00][D] remote disconnected, client num= 0


### Multi-Clients

In [4]:
import socket

try:
    port = 10020
    c1 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    c2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    c1.connect(('localhost', port))
    c2.connect(('localhost', port))
    
    c1.send(b'hello from client 1')
    data = c1.recv(1024)
    LOGD(data)
    
    c2.send(b'hello from client 2')
    data = c2.recv(1024)
    LOGD(data)
    
    c1.close()
    c2.close()
except Error as err:
    LOGE(err)


2020/05/17 21:12:32.319207 3424 [1.00][D] remote connected, client num= 1
2020/05/17 21:12:32.321207 3424 [1.00][D] remote connected, client num= 2
2020/05/17 21:12:32.321207 1480 [1.00][D] b'server received: hello from client 1'
2020/05/17 21:12:32.322207 1480 [1.00][D] b'server received: hello from client 2'
2020/05/17 21:12:32.324208 3424 [1.00][D] remote disconnected, client num= 1
2020/05/17 21:12:32.325208 3424 [1.00][D] remote disconnected, client num= 0


### Server + Client in the same select

In [ ]:
import socket, select, threading, time, traceback

port = 10021

# server
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server.bind(('localhost', port))
server.listen(5)

# client
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.setblocking(0)

# create other thread for testing
def client_thread():
    time.sleep(0.5)
    ret = client.connect_ex(('localhost', port))
    LOGD(ret)
    time.sleep(0.5)
    client.send(b'hello 1')
    time.sleep(0.5)
    client.send(b'hello 2')
    time.sleep(0.5)
    client.close()    
threading.Thread(target=client_thread).start()

# select
inputs = [server, client]

while True:
    try:
        readable, _, _ = select.select(inputs, [], [])
        for sck in readable:
            if sck is server:
                client, addr = sck.accept()
                client.setblocking(0)
                inputs.append(client)
                LOGD("remote connected, inputs num=", len(inputs))
            elif sck is client:
                try:
                    data = sck.recv(1024)
                    LOGD('client received the data from server:', data)
                except Exception as err: # if remote side close the connection
                    sck.close()
                    inputs.remove(sck)
                    LOGD("client is closed, inputs num=", len(inputs))
            else:
                try:
                    data = sck.recv(1024)
                    sck.send(b'server received: ' + data)
                except Exception as err: # if remote side close the connection
                    sck.close()
                    inputs.remove(sck)
                    LOGD("remote disconnected, inputs num=", len(inputs))
    except Exception as err:
        LOGE(traceback.format_exc())


2020/05/17 21:12:32.858238 4448 [1.00][D] 10035
2020/05/17 21:12:32.869239 1480 [1.00][D] remote connected, inputs num= 3
2020/05/17 21:12:33.369267 1480 [1.00][D] client received the data from server: b'server received: hello 1'
2020/05/17 21:12:33.869296 1480 [1.00][D] client received the data from server: b'server received: hello 2'
2020/05/17 21:12:34.369325 1480 [1.00][D] client is closed, inputs num= 2
2020/05/17 21:12:34.372325 1480 [1.00][D] remote disconnected, inputs num= 1
